In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("Heart Attack.csv")

In [5]:
labelencodercalss = preprocessing.LabelEncoder()
df["class"] = labelencodercalss.fit_transform(df["class"])

In [6]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3-Q1

lowerbound = Q1 - IQR *1.5
upperbound = Q3 + IQR *1.5

In [7]:
df_no_outlier = df[~((df<lowerbound)|(df>upperbound)).any(axis=1)]

In [8]:
df_no_outlier.shape

(789, 9)

In [14]:
X = df_no_outlier.drop(["class"],axis=1)
y = df_no_outlier["class"]

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)
modelgbr = GradientBoostingClassifier()
modelgbr.fit(X_train,y_train)
predictiongbr = modelgbr.predict(X_test)

In [16]:
print(classification_report(y_test,predictiongbr))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       173
           1       0.99      0.98      0.99       143

    accuracy                           0.99       316
   macro avg       0.99      0.99      0.99       316
weighted avg       0.99      0.99      0.99       316



In [17]:
import pickle
pickle.dump(modelgbr, open("model.pkl","wb"))